In [ ]:
import pandas as pd
import numpy as np
import math
import altair as alt

# enable correct rendering
alt.renderers.enable('default')
# uses intermediate json files to speed things up
alt.data_transformers.enable('json')

# New comment

DataTransformerRegistry.enable('json')

In [ ]:
#Importing Main Dataset
df = pd.read_csv('data2.csv', encoding='latin1')

In [ ]:

#Dataframe Cleanup
#Stockcode which contains only digits signifies sale entry
#Hence we will filter out enteries with digit only stockcode

clean_df = df[df.StockCode.str.contains('^\d', regex=True, na=False)]

# Drop quantities which are negative 
clean_df = clean_df[clean_df["Quantity"] >= 0]

#Adding SalesValue column to the Dataframe
clean_df["SalesValue"] = clean_df["Quantity"]*clean_df["UnitPrice"]

#UK Dataframe
uk_df = clean_df[clean_df['Country'] == 'United Kingdom']

#Rest of the World Dataframe
row_df = clean_df.drop(clean_df[clean_df['Country'] == 'United Kingdom'].index)

In [ ]:
#Grouping sales by customers
customer_df = clean_df.groupby(by=['CustomerID']).agg({'Quantity':['min', 'max', 'sum'], 'UnitPrice':['min', 'max', 'sum']})
print("Number of unique customers = ", customer_df.index.size)


Number of unique customers =  4335


In [ ]:
# Importing Secondary Datasets - Records for year range 2007 to 2011
# HDI could be additional variable
# Smartphone penetration dataset if available
# Credit card 

#Purchasing power parity GDP, PPP (constant 2017 international $) | Data (worldbank.org)
gdp_df = pd.read_excel('API_NY.GDP.MKTP.PP.KD_DS2_en_excel_v2_2764839.xls', 
                       sheet_name=0, header=3, usecols="A,B,D,AZ:BD")

#Inflation CPI Consumer price index (2010 = 100) | Data (worldbank.org)
cpi_df = pd.read_excel("API_FP.CPI.TOTL_DS2_en_excel_v2_2765329.xls",
                       sheet_name=0, header=3, usecols="A,B,D,AZ:BD")

#Debt % versus GDP External debt stocks, long-term (DOD, current US$) | Data (worldbank.org)
#extdebt_df = pd.read_excel("API_DT.DOD.DLXF.CD_DS2_en_excel_v2_2823747.xls",
#                           sheet_name=0, header=3, usecols="A,B,D,AZ:BD")

#Individuals using the Internet (% of population) 

internet_df = pd.read_excel("API_IT.NET.USER.ZS_DS2_en_excel_v2_2764008.xls",
                    sheet_name=0, header=3, usecols="A,B,D,AZ:BD")

#Exchange rate fluctuation (L5Y) Official exchange rate (LCU per US$, period average) | Data (worldbank.org)
exchrate_df = pd.read_excel("API_PA.NUS.FCRF_DS2_en_excel_v2_2764464.xls",
                            sheet_name=0, header=3, usecols="A,B,D,AZ:BD")

#Population Population, total | Data (worldbank.org)
pop_df = pd.read_excel("API_SP.POP.TOTL_DS2_en_excel_v2_2764317.xls",
                       sheet_name=0, header=3, usecols="A,B,D,AZ:BD")

#Merchandise imports Merchandise imports (current US$) | Data (worldbank.org)
merch_df = pd.read_excel("API_TM.VAL.MRCH.CD.WT_DS2_en_excel_v2_2766285.xls",
                         sheet_name=0, header=3, usecols="A,B,D,AZ:BD")

#CERDI Sea Distance Dataset
seadist_df = pd.read_excel("CERDI-seadistance.xlsx", usecols="A,B,C")
# Cleaning Sea Distance DF to only include entries with UK as the origin
seadist_df = seadist_df[seadist_df["iso1"]=="GBR"]


In [ ]:
# Function to normalize country names to code that will be used as a key to combine all datasets
def valeurs(k):
    filtered={'United Kingdom': 'GBR',
 'France': 'FRA',
 'USA': 'USA',
 'Belgium': 'BEL',
 'Australia': 'AUS',
 'EIRE': 'IRL',
 'Germany': 'DEU',
 'Portugal': 'PRT',
 'Japan': 'JPN',
 'Denmark': 'DNK',
 'Nigeria': 'NGA',
 'Netherlands': 'NLD',
 'Poland': 'POL',
 'Spain': 'ESP',
 'Channel Islands': 'CHI',
 'Italy': 'ITA',
 'Cyprus': 'CYP',
 'Greece': 'GRC',
 'Norway': 'NOR',
 'Austria': 'AUT',
 'Sweden': 'SWE',
 'United Arab Emirates': 'ARE',
 'Finland': 'FIN',
 'Switzerland': 'CHE',
 'Malta': 'MLT',
 'Bahrain': 'BHR',
 'Bermuda': 'BMU',
 'Hong Kong': 'HKG',
 'Singapore': 'SGP',
 'Thailand': 'THA',
 'Israel': 'ISR',
 'Lithuania': 'LTU',
 'Lebanon': 'LBN',
 'Korea': 'KOR',
 'Brazil': 'BRA',
 'Canada': 'CAN',
 'Iceland': 'ISL'}
    try:
        x=filtered[k]
    except:
        x=None
    return x

In [ ]:

# Function to get all the country related information for the analysis task
def countrydf(name, frequency="M"):
    # Country specific local dataframe
    cdf = clean_df.loc[clean_df["Country"] == name]

    # Datetime conversion
    cdf["date"] = pd.to_datetime(cdf.InvoiceDate)

    # Now lets find cummulative Sales for plot
    # First lets group by Date to get transaction total per day
    plotdf = cdf.set_index("date").resample(frequency)['SalesValue'].sum()
    # Convert series to dataframe
    plotdf = plotdf.to_frame()

    # Total sales for the country
    sales = plotdf["SalesValue"].sum()

    # DF that conforms to 13 row format if frequency is set to monthly
    if frequency == "M":
        plotdf = (plotdf + dummydf).fillna(0)
    
    # Number of unique customers in the country
    custcnt = len(cdf["CustomerID"].unique())

    # Numpy array of unique stock sold in each country
    uniquestock = cdf["StockCode"].unique() #add .tolist() if list output is desired
    
    #Country code
    code = valeurs(name)
    
    # GDP, CPI,  for year 2011
    gdp = gdp_df.loc[gdp_df["Country Code"] == code, "2011"].item()
    cpi = cpi_df.loc[cpi_df["Country Code"] == code, "2011"].item()
    
    #exchrate = exchrate_df.loc[exchrate_df["Country Code"] == code, "2011"].item()
    
    pop = pop_df.loc[pop_df["Country Code"] == code, "2011"].item()
    merch = merch_df.loc[merch_df["Country Code"] == code, "2011"].item()
    internet = internet_df.loc[internet_df["Country Code"] == code, "2011"].item()
    
    # Sea Distance
    dist = seadist_df.loc[seadist_df["iso2"]==code, "seadistance"].item()
    
    return {'name':name, 'code':code, 'df':plotdf, 'totalsales':sales, 'customercnt':custcnt, 
            'uniqueStockID': uniquestock, 'gdp':gdp, 'cpi':cpi, 'population':pop, 
            'merchsales': merch, 'internet':internet, 'distance':dist }


# Creating dummydf to obtain fixed 13 row plotdf
dummyindex = ['2010-12-31', '2011-01-31', '2011-02-28', '2011-03-31',
             '2011-04-30', '2011-05-31', '2011-06-30', '2011-07-31',
             '2011-08-31', '2011-09-30', '2011-10-31', '2011-11-30',
               '2011-12-31']
dummyvalues = [0,0,0,0,0,0,0,0,0,0,0,0,0]
dummydf = pd.DataFrame({'date':dummyindex, 'SalesValue':dummyvalues})
dummydf = dummydf.set_index('date')


In [ ]:
# Creating Final DF that will be used for regression analysis
# Safe to ignore the SettingWithCopyWarning warning
countries = ['France', 'USA', 'Belgium', 'EIRE', 'Germany', 'Portugal', 'Japan', 'Denmark', 'Nigeria', \
    'Netherlands', 'Poland', 'Spain', 'Italy', 'Cyprus', 'Greece','Norway', 'Austria', 'Sweden', \
        'United Arab Emirates', 'Finland', 'Switzerland', 'Malta', 'Bahrain', 'Bermuda', 'Hong Kong', \
            'Singapore', 'Thailand', 'Israel', 'Lithuania', 'Lebanon', 'Korea', 'Brazil', 'Canada', 'Iceland']

# Creating list of dictionaries obtained using countrydf function
finallist = [countrydf(country) for country in countries]

# Creating Dataframe from that list
finaldf = pd.DataFrame(finallist)

finaldf.head(3)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,name,code,df,totalsales,customercnt,uniqueStockID,gdp,cpi,population,merchsales,internet,distance
0,France,FRA,SalesValue date ...,184582.74,88,"[22728, 22727, 22726, 21724, 21883, 10002, 217...",2.800855e+12,102.111598,65342789.0,7.200280e+11,77.819999,3224.950
1,USA,USA,SalesValue date ...,3580.39,4,"[22722, 22979, 84987, 22720, 22993, 47580, 229...",1.706195e+13,103.156842,311583481.0,2.266024e+12,69.729461,7767.240
2,Belgium,BEL,SalesValue date ...,36927.34,25,"[84375, 21217, 21212, 21977, 22417, 21975, 219...",5.315359e+11,103.532082,11038264.0,4.669430e+11,81.609996,323.696


In [ ]:
#Example of how to extract Monthly Sales data for each country
# In this example, we will use Canada and for that code is "CAN"
example = finaldf.loc[finaldf["code"]=='CAN', 'df'].item()
example

,SalesValue
date,
2011-03-31 00:00:00,0.0
2011-04-30 00:00:00,0.0
2011-05-31 00:00:00,0.0
2011-06-30 00:00:00,0.0
2011-07-31 00:00:00,0.0
2011-08-31 00:00:00,0.0
2010-12-31,0.0
2011-01-31,0.0
2011-02-28,0.0


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=023a0e03-cc30-4d6d-b448-6d27799cca93' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>